Notebook slightly adapted from [colab@google](https://colab.research.google.com/notebooks/gpu.ipynb)

The goal of this notebook is to test the GPU and perform a very simple (and fast) benchmark.

In [5]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Expected Output:**
   
   <html>
<table style="width:30%">
   <tr>
    <th>Message</th>
  </tr>
  <tr>
    <th>Found GPU at: /device:GPU:0</th>
  </tr>
</table> 
</html>



In [4]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
    random_image_cpu = tf.random_normal((24, 128, 128, 3))
    net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7, padding='valid')
    net_cpu = tf.layers.conv2d(net_cpu, 16, 3)
    net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
    random_image_gpu = tf.random_normal((24, 128, 128, 3))
    net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7, padding='valid')
    net_gpu = tf.layers.conv2d(net_gpu, 16, 3)
    net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
    sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
    print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
    raise

def cpu():
    sess.run(net_cpu)

def gpu():
    sess.run(net_gpu)

    
# Runs the op several times.
print('Time (s) to convolve filters over random images '
      '(batch x height x width x channel). Sum of 20 runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=20, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=20, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x1000x1000x3 images (batch x height x width x channel). Sum of 20 runs.
CPU (s):
4.767332523235353
GPU (s):
0.2375035052317287
GPU speedup over CPU: 20x
